# Import

In [1]:
import os

import pandas as pd
import numpy as np

from tqdm import tqdm

from sklearn.model_selection import train_test_split

import torch
from torch.utils.data import DataLoader, Subset

import torch.nn.functional as F
from torch import nn, optim

from sklearn.metrics import log_loss

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

from config.config import load_config
from utils.utils import *
from datasets import get_dataset
from datasets.transforms import build_transforms
from models import get_model
from trainer.training import training
from trainer.evaluation import evaluation
from trainer.train_loop import training_loop
from utils.EarlyStopping import EarlyStopping
from utils.scheduler_factory import get_scheduler

Using device: cuda


c:\Users\fkjy1\anaconda3\envs\hai\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Hyperparameter Setting

In [2]:
cfg = load_config("config/main_config.yaml")
train_transform, val_transform = build_transforms(cfg["transforms"]["train"]), build_transforms(cfg["transforms"]["val"])

DatasetClass = get_dataset(cfg['DATASET'])
ModelClass = get_model(cfg['MODEL'])
cfg_scheduler = cfg["scheduler"]

# Fixed RandomSeed

In [3]:
set_seed(cfg['SEED'])

seed 고정 완료!


# Data Load

In [4]:
data_path = './data'
train_root = f'{data_path}/train'
test_root = f'{data_path}/test'
output_root = './output'
makedirs(output_root)

폴더 확인 완료!!


In [5]:
# 전체 데이터셋 로드
full_dataset = DatasetClass(train_root, transform=None)
print(f"총 이미지 수: {len(full_dataset)}")

targets = [label for _, label in full_dataset.samples]

class_names = full_dataset.classes

# Stratified Split
train_idx, val_idx = train_test_split(
    range(len(targets)), test_size=0.2, stratify=targets, random_state=42
)

# Subset + transform 각각 적용
train_dataset = Subset(DatasetClass(train_root, transform=train_transform), train_idx)
val_dataset = Subset(DatasetClass(train_root, transform=val_transform), val_idx)
print(f'train 이미지 수: {len(train_dataset)}, valid 이미지 수: {len(val_dataset)}')


# DataLoader 정의
train_loader = DataLoader(train_dataset, batch_size=cfg['BATCH_SIZE'], shuffle=True, num_workers=os.cpu_count() // 2, pin_memory=True, persistent_workers=True)
val_loader = DataLoader(val_dataset, batch_size=cfg['BATCH_SIZE'], shuffle=False, num_workers=os.cpu_count() // 2, pin_memory=True, persistent_workers=True)

총 이미지 수: 33137
train 이미지 수: 26509, valid 이미지 수: 6628


# Train/ Validation

In [6]:
model = ModelClass(num_classes=len(class_names)).to(device)
# best_logloss = float('inf')

early_stopping = EarlyStopping(patience=cfg['patience'], verbose=True, save_path=f'{output_root}/checkpoint.pth')

# 손실 함수
criterion = nn.CrossEntropyLoss()

# 옵티마이저
optimizer = optim.Adam(model.parameters(), lr=cfg['LEARNING_RATE'], weight_decay=0)

# 스케쥴러
Scheduler = get_scheduler(cfg_scheduler['name'], optimizer, cfg_scheduler['params'])

model, valid_max_accuracy = training_loop(model, train_loader, val_loader, train_dataset, val_dataset, criterion, optimizer, device, cfg['EPOCHS'], early_stopping, class_names)

# # 학습 및 검증 루프
# for epoch in range(cfg['EPOCHS']):
    
#     model, train_loss, train_accuracy = training(model, train_loader, train_dataset, criterion, optimizer, device, epoch, cfg['EPOCHS'])
    
#     model, valid_loss, valid_accuracy, val_logloss = evaluation(model, val_loader, val_dataset, criterion, device, epoch, cfg['EPOCHS'], class_names)

#     # 결과 출력
#     print(f"Train Loss : {train_loss:.4f} || Valid Loss : {valid_loss:.4f} | Valid Accuracy : {valid_accuracy:.4f}%")

#     # Best model 저장
#     if val_logloss < best_logloss:
#         best_logloss = val_logloss
#         torch.save(model.state_dict(), f'{output_root}/best_model.pth')
#         print(f"📦 Best model saved at epoch {epoch+1} (logloss: {val_logloss:.4f})")

c:\Users\fkjy1\anaconda3\envs\hai\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\fkjy1\anaconda3\envs\hai\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Epoch [1/1], Valid Loss: 4.0765: 100%|██████████| 104/104 [00:43<00:00,  2.40it/s]
c:\Users\fkjy1\anaconda3\envs\hai\Lib\site-packages\sklearn\metrics\_classification.py:3001: UserWarning: The y_pred values do not sum to one. Make sure to pass probabilities.
  warnings.warn(


  ✅ Validation loss improved. Saving model...
Epoch [1/1], Train Loss: 5.4127, Train Accuracy: 0.0683 Valid Loss: 4.0421, Valid Accuracy: 0.2526


# Inference

In [ ]:
test_dataset = DatasetClass(test_root, transform=val_transform, is_test=True)
test_loader = DataLoader(test_dataset, batch_size=cfg['BATCH_SIZE'], shuffle=False)

In [9]:
# 저장된 모델 로드
model = ModelClass(num_classes=len(class_names))
model.load_state_dict(torch.load('best_model.pth', map_location=device))
model.to(device)

# 추론
model.eval()
results = []

with torch.no_grad():
    for images in test_loader:
        images = images.to(device)
        outputs = model(images)
        probs = F.softmax(outputs, dim=1)

        # 각 배치의 확률을 리스트로 변환
        for prob in probs.cpu():  # prob: (num_classes,)
            result = {
                class_names[i]: prob[i].item()
                for i in range(len(class_names))
            }
            results.append(result)
            
pred = pd.DataFrame(results)

# Submission

In [10]:
submission = pd.read_csv(f'{data_path}/sample_submission.csv', encoding='utf-8-sig')

# 'ID' 컬럼을 제외한 클래스 컬럼 정렬
class_columns = submission.columns[1:]
pred = pred[class_columns]

submission[class_columns] = pred.values
submission.to_csv('baseline_submission.csv', index=False, encoding='utf-8-sig')